<a href="https://colab.research.google.com/github/JuanC-Fandino/RedClasificacionResiduos/blob/master/RedClasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías

In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import itertools

# Obtención del Dataset

In [2]:
!git clone https://github.com/JuanC-Fandino/RedClasificacionResiduos.git

Cloning into 'RedClasificacionResiduos'...
remote: Enumerating objects: 4543, done.
remote: Total 4543 (delta 0), reused 0 (delta 0), pack-reused 4543
Receiving objects: 100% (4543/4543), 159.82 MiB | 38.31 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (4401/4401), done.


# Creación de los sets de entrenamiento y validación

In [3]:
DIR = "/content/RedClasificacionResiduos/dataset"
alto_img = 256
ancho_img = 256
batch_size = 64
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(DIR, validation_split=0.2, subset="training", batch_size=batch_size, smart_resize=True, image_size=(alto_img, ancho_img), seed=25)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(DIR, validation_split=0.2, subset="validation", batch_size=batch_size, smart_resize=True, image_size=(alto_img, ancho_img), seed=25)

clases = train_dataset.class_names
total_clases = len(train_dataset.class_names)
print(clases)

Found 4396 files belonging to 8 classes.
Using 3517 files for training.
Found 4396 files belonging to 8 classes.
Using 879 files for validation.
['Aluminio', 'Carton', 'Envases_Plastico', 'Organicos', 'Papel', 'Plasticos', 'Tetra_Pak', 'Vidrio']


# Optimización del flujo de datos


In [4]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Entrenamiento del modelo

In [6]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(alto_img,
                                  ancho_img,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(alto_img, ancho_img, 3)),
  layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.25),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(total_clases)
])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        

model.compile(tf.keras.optimizers.Adam(learning_rate=0.00125),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs=50
history = model.fit(
  train_dataset,
  validation_data=test_dataset,
  epochs=epochs,
  callbacks = [early_stopping]
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

Epoch 1/50
55/55 [==============================] - 28s 243ms/step - loss: 1.9864 - accuracy: 0.2462 - val_loss: 1.7774 - val_accuracy: 0.3015
Epoch 2/50
55/55 [==============================] - 13s 218ms/step - loss: 1.6450 - accuracy: 0.3620 - val_loss: 1.5260 - val_accuracy: 0.4278
Epoch 3/50
55/55 [==============================] - 13s 215ms/step - loss: 1.4452 - accuracy: 0.4635 - val_loss: 1.4708 - val_accuracy: 0.4653
Epoch 4/50
55/55 [==============================] - 13s 215ms/step - loss: 1.3157 - accuracy: 0.5061 - val_loss: 1.4372 - val_accuracy: 0.4676
Epoch 5/50
55/55 [==============================] - 13s 215ms/step - loss: 1.2032 - accuracy: 0.5502 - val_loss: 1.3942 - val_accuracy: 0.5017
Epoch 6/50
55/55 [==============================] - 13s 215ms/step - loss: 1.0423 - accuracy: 0.6233 - val_loss: 1.3452 - val_accuracy: 0.5267
Epoch 7/50
55/55 [==============================] - 13s 216ms/step - loss: 0.8692 - accuracy: 0.6830 - val_loss: 1.4506 - val_accuracy: 0.5301

KeyboardInterrupt: ignored

In [ ]:
 model.save("residuos_model.h5")

!pip install tensorflowjs
!tensorflowjs_converter --input_format keras "residuos_model.h5" ./jsmodel --output_format=tfjs_graph_model 

# Guardar modelo

In [ ]:
!zip -r /content/model.zip /content/jsmodel

In [ ]:
model.summary()

# Evaluación del Modelo

In [ ]:
# Precisión
plt.plot(range(0, epochs), acc, color="b", label="Accuracy")
plt.plot(range(0, epochs), val_acc, color="r", label="Test Accuracy")
plt.legend()
plt.show()

In [ ]:
# Perdida
plt.plot(range(0, epochs), loss, color="b", label="Loss")
plt.plot(range(0, epochs), val_loss, color="r", label="Test Loss")
plt.legend()
plt.show()

In [ ]:
# Matriz de confusión
def plot_confusion_matrix(cm, target_names, cmap=None):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title('Confusion matrix')
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}%; misclass={:0.4f}%'.format(accuracy, misclass))
    plt.show()

plt.figure(figsize=(10, 10))
true = []
predictions = []

for i in os.listdir(DIR):
  folderPath = os.path.join(DIR, i)
  for j in os.listdir(folderPath)[:550]:
    fullPath = os.path.join(folderPath, j)
    try:
      img = tf.keras.preprocessing.image.load_img(fullPath, target_size=(256, 256))
      img_array = tf.keras.preprocessing.image.img_to_array(img)
      img_array = tf.expand_dims(img_array, 0) 

      preds = model.predict(img_array)
      true.append(clases.index(i))
      predictions.append(np.argmax(preds))
    except:
      print("Error on image:", fullPath)

plot_confusion_matrix(tf.math.confusion_matrix(true, predictions), clases)